In [ ]:
import json
import os
from utils.data_dirs import data_dirs
from utils.filter_road import filter_road

In [ ]:
d = data_dirs()
print(d.raw_road)
print(d.processed_road)
print(d.raw_map)
print(d.processed_map)

In [ ]:
raw_expansion = os.path.join(d.raw_map, 'expansion')
processed_expansion = os.path.join(d.processed_map, 'expansion')

In [ ]:
raw_expansion

In [ ]:
processed_expansion

In [ ]:
int('f', 16)

In [ ]:
EXAMPLE_TOKEN = '0073298b-b2f4-4f89-97cd-4241a1599831'

def is_hex(h: 'str'):
    try:
        int(h, 16)
        return True
    except:
        return False

def is_token(token) -> "bool":
    if not isinstance(token, str):
        return False

    chunks = token.split('-')
    if len(chunks) != 5:
        return False
    
    if [len(c) for c in chunks] != [8, 4, 4, 4, 12]:
        return False
    
    return all(map(is_hex, chunks))

In [ ]:
import itertools

[*itertools.chain([1,2], [4], itertools.chain(*(x for x in [[1], [2]])), map(lambda x: x, [2]))]

In [ ]:
import itertools

def extract_all_tokens(o):
    if isinstance(o, dict):
        return itertools.chain(itertools.chain(*map(extract_all_tokens, o.values())), itertools.chain(*map(extract_all_tokens, o.keys())))
    elif isinstance(o, list) or isinstance(o, tuple):
        return itertools.chain(*map(extract_all_tokens, o))
    elif is_token(o):
        return [o]
    else:
        return []

def extract_all_non_tokens(o):
    if isinstance(o, dict):
        return itertools.chain(*map(extract_all_non_tokens, o.values()))
    elif isinstance(o, list):
        return itertools.chain(*map(extract_all_non_tokens, o))
    elif is_token(o):
        return []
    else:
        if not (isinstance(o, int) or isinstance(o, float)):
            return [o]
        return []

In [ ]:
all_non_tokens = set()
tokens = dict()
for e in os.listdir(raw_expansion):
    with open(os.path.join(raw_expansion, e), "r") as f:
        e_json = json.load(f)
    key = e.split('.')[0]
    tokens[key] = {*extract_all_tokens(e_json)}
    all_non_tokens.update(extract_all_non_tokens(e_json))

print(all_non_tokens)
print(tokens.keys())

In [ ]:
_tokens = []
for e in os.listdir(raw_expansion):
    with open(os.path.join(raw_expansion, e), "r") as f:
        e_json = json.load(f)
    
#     tokens: "Set[str]" = set()
        
#     for k, value in e_json.items():
#         if type(value) == list:
#             tokens.update({v["token"] for v in value if type(v) == dict})
#         elif type(value) == dict:
#             tokens.update(value.keys())
    tokens = {*extract_all_tokens(e_json)}
    
    if not os.path.exists(processed_expansion):
        os.makedirs(processed_expansion)
    
    with open(os.path.join(processed_expansion, e[:-len(".json")] + "-tokens.json"), "w") as f:
        json.dump([*tokens], f)
    
    _tokens.append(tokens)

for t1 in _tokens:
    for t2 in _tokens:
        if t1 == t2: continue
        print(t1 & t2)

In [ ]:
for tokens_f in os.listdir(processed_expansion):
    with open(os.path.join(processed_expansion, tokens_f), "r") as f:
        tokens = {*json.load(f)}
    
    region = tokens_f[:-len("-tokens.json")]
    print(region)
    filtered = filter_road(d.raw_road, tokens)
    for k, v in filtered.items():
        print(k, len(v))
        base = os.path.join(d.processed_road, region)
        if not os.path.exists(base):
            os.makedirs(base)
        with open(os.path.join(base, k + ".json"), "w") as f:
            json.dump(v, f, indent=1)
    

In [ ]:
base_old = os.path.join(d.processed_road, "boston-old")
base_seaport = os.path.join(d.processed_road, "boston-seaport")

def remove_heading(d):
    if 'heading' in d:
        del d['heading']
    return d

for e in os.listdir(base_old):
    with open(os.path.join(base_old, e), "r") as f:
        old = json.load(f)
    with open(os.path.join(base_seaport, e), "r") as f:
        seaport = json.load(f)
    
    print(e)
    old_set = set((json.dumps(o, sort_keys=True) for o in map(remove_heading, old)))
    seaport_set = set((json.dumps(o, sort_keys=True) for o in map(remove_heading, seaport)))
    print('boston-old is subset:', len(old_set - seaport_set) == 0)
    print()
    

In [ ]:
with open(os.path.join(base_old, 'polygon.json'), "r") as f:
    old = json.load(f)
with open(os.path.join(base_seaport, 'polygon.json'), "r") as f:
    seaport = json.load(f)
# old[0]
print([o for o in old if o['id'] == "971af7a8-855a-474a-8715-f6b8016e78ff"])
print([o for o in seaport if o['id'] == "971af7a8-855a-474a-8715-f6b8016e78ff"])

In [ ]:
with open(os.path.join(base_old, 'segment.json'), "r") as f:
    old = json.load(f)
with open(os.path.join(base_seaport, 'segment.json'), 'r') as f:
    seaport = json.load(f)
print(json.dumps([o for o in old if o['polygonId'] == "971af7a8-855a-474a-8715-f6b8016e78ff"], indent=2))
print(json.dumps([o for o in seaport if o['polygonId'] == "971af7a8-855a-474a-8715-f6b8016e78ff"], indent=2))